In [1]:
import pandas as pd
import spacy

In [2]:
#url = 'https://raw.githubusercontent.com/DNason1999/simple_repository/master/df_merged.csv'
url = './Data/best_merge.csv'
df = pd.read_csv(url)

In [3]:
df = df[['id', 'name', 'race', 'Description', 'Rating', 'flavors', 'positive', 'negative', 'medical']]

In [4]:
target = list(df.index)

In [5]:
df = df.set_index('id')

In [6]:
df.shape

(2826, 8)

In [7]:
df.head()

,name,race,Description,Rating,flavors,positive,negative,medical
id,,,,,,,,
1,Afpak,hybrid,"Afpak, named for its direct Afghani and Pakist...",4.2,"['Earthy', 'Chemical', 'Pine', 'Spicy/Herbal']","['Relaxed', 'Hungry', 'Happy', 'Sleepy', 'Crea...",['Dizzy'],"['Depression', 'Insomnia', 'Pain', 'Stress', '..."
2,African,sativa,African refers to the indigenous varieties of ...,3.9,"['Spicy/Herbal', 'Pungent', 'Earthy', 'Pepper']","['Euphoric', 'Happy', 'Creative', 'Energetic',...",['Dry Mouth'],"['Depression', 'Pain', 'Stress', 'Lack of Appe..."
3,Afternoon Delight,hybrid,"Afternoon Delight, created by Colorado Seed In...",4.8,"['Pepper', 'Flowery', 'Pine', 'Pungent', 'Citr...","['Relaxed', 'Hungry', 'Euphoric', 'Uplifted', ...","['Dizzy', 'Dry Mouth', 'Paranoid']","['Depression', 'Insomnia', 'Pain', 'Stress', '..."
4,Afwreck,hybrid,Afwreck is a hybrid cross of Afghani and Train...,4.2,"['Pine', 'Earthy', 'Flowery', 'Pungent']","['Relaxed', 'Happy', 'Creative', 'Uplifted', '...","['Dizzy', 'Dry Mouth', 'Paranoid', 'Dry Eyes']","['Pain', 'Stress', 'Headache', 'Fatigue', 'Hea..."
5,Agent Orange,hybrid,Don’t let the name scare you! The only herbici...,4.2,"['Citrus', 'Orange', 'Sweet', 'Earthy']","['Relaxed', 'Euphoric', 'Happy', 'Energetic', ...","['Dizzy', 'Dry Mouth', 'Paranoid', 'Dry Eyes']","['Depression', 'Pain', 'Stress', 'Nausea', 'He..."


In [8]:
texts = []

for index, row in df.iterrows():
    s = ''
    for item in row[1:4]:
        if(type(item) == str):
            s += item
        elif(type(item) == float):
            s += str(item)
        s += ' '
    for item in row[6:]:
        if type(item) == float:
            pass
        else:
            for subitem in eval(item):
                s += str(subitem)
                s+= ' '
            s += ' '
    s = s.replace('/',' ').replace('  ', ' ')
                
    texts.append(s)

In [9]:
df['text'] = texts

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', 
                        ngram_range=([1,1]),
                        max_df=.97,
                        min_df=3,)

dtm = tfidf.fit_transform(df['text'])

In [11]:
dtm.shape

(2826, 3576)

In [12]:
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

In [13]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree', leaf_size=100)
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=100, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [81]:
# import pickle

# pickle.dump(tfidf, open('./Models/tfidf.pkl', 'wb'))

# pickle.dump(nn, open('./Models/nn.pkl', 'wb'))

In [14]:
sample_text = """
I experience chronic widespread pain that causes my joints, skin, head, and digestive system to hurt, so I need strains that provide significant pain relief. I would also like to find a sleep aid for my insomnia. Depression makes it difficult to wake and get things done, and occasionally, depressive episodes interfere greatly with my function. 
"""

In [15]:
sample_text_tokens = tfidf.transform([sample_text]).todense()

In [16]:
sample_text_tokens

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [17]:
predictions = nn.kneighbors(sample_text_tokens)[1][0]

In [18]:
strains = df.iloc[predictions]

In [19]:
strains

,name,race,Description,Rating,flavors,positive,negative,medical,text
id,,,,,,,,,
936,Green Door Kush,hybrid,Green Door Kush is a hybrid strain that is kno...,4.5,"['Citrus', 'Spicy/Herbal', 'Woody']","['Relaxed', 'Euphoric', 'Happy', 'Creative', '...","['Dry Mouth', 'Dry Eyes', 'Anxious']","['Insomnia', 'Pain', 'Stress', 'Nausea', 'Musc...",hybrid Green Door Kush is a hybrid strain that...
2549,Khola,hybrid,"Khola is a sociable cross between Brazilian, T...",4.2,[],[],NaN,NaN,hybrid Khola is a sociable cross between Brazi...
1612,Purple Arrow,hybrid,"When it comes to knocking out pain, no medical...",4.2,"['Earthy', 'Sweet', 'Citrus']","['Happy', 'Energetic', 'Uplifted', 'Sleepy', '...","['Dizzy', 'Dry Mouth', 'Paranoid', 'Dry Eyes']","['Depression', 'Pain', 'Stress', 'Headache', '...","hybrid When it comes to knocking out pain, no ..."
2237,Avi Dekel,hybrid,Avi-Dekel is a sativa-dominant strain develope...,4.5,[],[],NaN,NaN,hybrid Avi-Dekel is a sativa-dominant strain d...
602,Cornbread,indica,Cornbread by Rare Dankness is an 80/20 indica-...,4.6,"['Lemon', 'Citrus', 'Earthy', 'Woody']","['Relaxed', 'Hungry', 'Euphoric', 'Happy', 'Sl...","['Dry Mouth', 'Dry Eyes']","['Depression', 'Insomnia', 'Pain', 'Stress', '...",indica Cornbread by Rare Dankness is an 80 20 ...


In [20]:
import requests

In [31]:
url = 'https://med-cabinet-4-api.herokuapp.com/predict'

payload = {
    "positive_effect" : ["euphoria"],
    "negative_effect" : ["sleeplessness, sickness"],
    "medical_effect" : ["pain"],
    "flavor" : ["earthy"],
    "desc" : ["A very fancy strain"],
    "min_rating" : [1.79],
    "num_resp" : 3
}

r = requests.request('POST', url, params=payload)

In [32]:
r

<Response [200]>

In [33]:
r.json()

{'description': 'The colorful hues and sweet flavors of Blueberry blend together with the resin production and heartiness of California Hash Plant to create Blue Hash, or Blueberry Hash. The aroma is a mixture of fresh berries with a pungent, earthy finish. Blue Hash produces a powerfully relaxing effect that is a great way to unwind at the end of a long day or at night before bedtime. Medical patients will appreciate Blue Hash’s ability to combat insomnia and suppress stress.',
 'flavor': "['Blueberry', 'Sweet', 'Earthy']",
 'id': 264,
 'medical': "['Depression', 'Insomnia', 'Pain', 'Stress', 'Lack of Appetite', 'Headache']",
 'name': 'Blue Hash',
 'negative': "['Dizzy', 'Dry Mouth', 'Paranoid', 'Dry Eyes']",
 'positive': "['Relaxed', 'Hungry', 'Happy', 'Tingly', 'Sleepy', 'Euphoric', 'Uplifted']",
 'race': 'hybrid',
 'rating': '4.4'}